#  **Dataleakage**

Jason Brownlee - Data Preparation for Machine Learning - Data Cleaning, Feature Selection, and Data (2020, machine learning mastery)

Chapter 4
 Data Preparation Without Data Leakage

## train-test split

In [18]:
# Step 1: Bibliotheken importieren
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 2: Datensatz definieren
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)

# Step 3: Split in Trainings- und Testdaten (vor Data Preparation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# Step 4: Datenvorbereitung (Scaler nur auf die Trainingsdaten anwenden)
scaler = MinMaxScaler()

# Den Scaler nur auf die Trainingsdaten fitten und anwenden
X_train = scaler.fit_transform(X_train)

# Die gleichen Transformationen auf die Testdaten anwenden (ohne erneut zu fitten)
X_test = scaler.transform(X_test)

# Step 5: Modell trainieren
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 6: Vorhersagen treffen
yhat = model.predict(X_test)

# Step 7: Vorhersagen bewerten
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.3f %%' % (accuracy*100))


Accuracy: 85.152 %


In [19]:
#
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5,
random_state=7)
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# define the scaler
scaler = MinMaxScaler()
# fit on the training dataset
scaler.fit(X_train)
# scale the training dataset
X_train = scaler.transform(X_train)
# scale the test dataset
X_test = scaler.transform(X_test)
# fit the model
model = LogisticRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % (accuracy*100))

Accuracy: 85.152


## k-fold Cross-Validatio

In [24]:
# Import von mean und std aus numpy zur Berechnung der mittleren Genauigkeit und Standardabweichung der Ergebnisse
from numpy import mean
from numpy import std

# Import der Funktion zur Erzeugung eines synthetischen Klassifikationsdatensatzes
from sklearn.datasets import make_classification

# Import der Funktionen für Cross-Validation und die Wiederholung von K-Fold-Verfahren
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Import des MinMaxScalers zur Skalierung der Features auf einen bestimmten Wertebereich (normalerweise zwischen 0 und 1)
from sklearn.preprocessing import MinMaxScaler

# Import der logistischen Regression, die als Modell verwendet wird
from sklearn.linear_model import LogisticRegression

# Import der Pipeline zur Kombination mehrerer Schritte der Vorverarbeitung und Modellierung
from sklearn.pipeline import Pipeline

# Schritt 1: Datensatz definieren
# Hier wird ein synthetischer Datensatz mit 1000 Datenpunkten und 20 Features erstellt
# - n_informative=15 bedeutet, dass 15 Features tatsächlich Informationen enthalten, die zur Klassifizierung nützlich sind
# - n_redundant=5 bedeutet, dass 5 Features eine lineare Kombination der informativen Features sind (sie sind also redundant)
# - random_state=7 sorgt für Reproduzierbarkeit
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)

# Schritt 2: Pipeline definieren
# Die Pipeline wird verwendet, um mehrere Verarbeitungsschritte in einem einzigen Modell zu kombinieren
steps = list()  # Liste von Schritten initialisieren

# Schritt 2.1: MinMaxScaler zur Liste hinzufügen
# Der MinMaxScaler wird verwendet, um die Features auf einen Bereich von 0 bis 1 zu skalieren.
steps.append(('scaler', MinMaxScaler()))

# Schritt 2.2: Logistische Regression zur Liste hinzufügen
# Hier wird das Modell definiert, in diesem Fall eine logistische Regression, die für binäre Klassifikation verwendet wird.
steps.append(('model', LogisticRegression()))

# Schritt 2.3: Die Pipeline mit den definierten Schritten erstellen
# - Die Pipeline führt zuerst die Skalierung der Daten durch (MinMaxScaler) und wendet danach die logistische Regression an.
pipeline = Pipeline(steps=steps)

# Schritt 3: Evaluationsverfahren definieren (Cross-Validation)
# RepeatedStratifiedKFold wird verwendet, um eine stratifizierte K-Fold-Cross-Validation mehrfach durchzuführen.
# - n_splits=10 bedeutet, dass der Datensatz in 10 Teile (Folds) aufgeteilt wird.
# - n_repeats=3 bedeutet, dass der Cross-Validation-Prozess 3-mal wiederholt wird.
# - random_state=1 sorgt dafür, dass die Ergebnisse reproduzierbar sind.
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Schritt 4: Modell evaluieren (Cross-Validation)
# cross_val_score führt die Cross-Validation auf der Pipeline aus und bewertet das Modell anhand des "accuracy" Scores.
# - Die Pipeline enthält alle Schritte (Skalierung und Modell)
# - X und y sind die Daten und Labels.
# - scoring='accuracy' bedeutet, dass die Genauigkeit als Bewertungsmaßstab verwendet wird.
# - cv=cv gibt das Evaluationsverfahren an (in diesem Fall RepeatedStratifiedKFold)
# - n_jobs=-1 bedeutet, dass alle verfügbaren CPU-Kerne genutzt werden, um die Berechnung parallel durchzuführen.
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# Schritt 5: Modellleistung berichten
# - mean(scores) berechnet die durchschnittliche Genauigkeit aus allen Cross-Validation-Ergebnissen.
# - std(scores) berechnet die Standardabweichung der Genauigkeit, um zu zeigen, wie stabil die Ergebnisse sind.
# - Die Ergebnisse werden als Prozentsatz ausgegeben (daher die Multiplikation mit 100).
print('Accuracy: %.3f (%.3f)' % (mean(scores) * 100, std(scores) * 100))


Accuracy: 85.400 (3.489)


In [22]:
#
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5,
random_state=7)
# define the pipeline
steps = list()
steps.append(('scaler', MinMaxScaler()))
steps.append(('model', LogisticRegression()))
pipeline = Pipeline(steps=steps)
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model using cross-validation
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores)*100, std(scores)*100))

Accuracy: 85.400 (3.489)
